# Code Generation and Evaluation

In [1]:
from human_eval.data import read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from datasets import load_dataset
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full,
                      reflection_prompt_basic, reflection_prompt, reflection_prompt_full, )
from helpers import get_tokenizer, get_model, clean_code, clean_code_light, is_function, write_jsonl

In [3]:
# select model, propmt, temperature, top_p
models = [
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    #('open-codestral-mamba', 'codestral_mamba'),
    #('open-mistral-7b', 'mistral_7b'),    
]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# reflection prompts & their names for log file name
reflection_prompts_and_names = [
    ( reflection_prompt_basic, 'reflection_prompt_basic' ),
    ( reflection_prompt, 'reflection_prompt' ),
    ( reflection_prompt_full, 'reflection_prompt_full' ), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx            = 2
prompt_idx           = 0
relection_prompt_idx = 0
temperature_idx      = 0
top_p_idx            = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
my_reflection_prompt, my_reflection_prompt_label = reflection_prompts_and_names[ relection_prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:             {model_name}')
print(f'Model nickname:         {model_nickname}')
print(f'Prompt name:            {my_prompt_label}')
print(f'Relfection prompt name: {my_reflection_prompt_label}')
print(f'\nPrompt:\n{my_prompt.strip().format("Starter Code")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:             open-mistral-nemo
Model nickname:         nemo
Prompt name:            complete_code_prompt_basic
Relfection prompt name: reflection_prompt_basic

Prompt:
Complete the following Python code:
Starter Code

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [4]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-06-16 04:35:35,508  INFO  *****     JOB CONFIGURATION     *****
2025-06-16 04:35:35,509  INFO  MODEL: open-mistral-nemo
2025-06-16 04:35:35,509  INFO  MODEL NICKNAME: nemo
2025-06-16 04:35:35,510  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-06-16 04:35:35,510  INFO  GENERATED CODE SAVED IN: logs_local/nemo_complete_code_prompt_basic_temperature1.0_topP1.0_completions_20250616_043535_5084.jsonl
2025-06-16 04:35:35,511  INFO  THIS LOG FILE: logs_local/logs/nemo_complete_code_prompt_basic_temperature1.0_topP1.0_log_20250616_043535_5084.log
2025-06-16 04:35:35,511  INFO  PROMPT:
Complete the following Python code:
Starter Code

2025-06-16 04:35:35,511  INFO  *****     END OF JOB CONFIGURATION     *****


In [5]:
# read dataset
file = 'data/Big_Code_Bench_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 1:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: BigCodeBench/0 <class 'str'>

task_id:
BigCodeBench/0

test:


import unittest
from unittest.mock import patch
from random import seed, shuffle
import itertools
class TestCases(unittest.TestCase):
    def test_default_numbers(self):
        # Test with default number range (1 to 10) to check that the result is a positive float.
        result = task_func()
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_custom_list(self):
        # Test with a custom list of small positive integers to ensure proper handling and positive result.
        result = task_func([1, 2, 3])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_negative_numbers(self):
        # Test with negative numbers to verify the function handles and returns a positive result.
        result = task_func([-3, -2, -1])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 

In [6]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [7]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [8]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-06-16 04:35:53,422  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It has been the state capital since 1854. Sacramento is known for its historic sites, such as the California State Capitol, Old Sacramento, and the Crocker Art Museum. It's also the sixth-largest city in California, with a population of around 500,000 people.


In [9]:
num_samples_per_task = 1
completions  = []

for task_id, task_body in tasks.items():
    logging.info( task_id )
    #if task_id == 'BigCodeBench/7':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        
        # generate completion
        full_prompt       = my_prompt.lstrip().format( task_body['prompt'] )
        proposed_solution = generate_response_with_backoff( model_name, full_prompt )
        
        # improve solution
        new_full_prompt   = my_reflection_prompt.lstrip().format(full_prompt, proposed_solution)
        improved_solution = generate_response_with_backoff( model_name, new_full_prompt )
        
        # check if improved solution is a Python function. If not, use the previously proposed solution
        cleaned_improved_solution = clean_code_light(improved_solution).strip()
        if is_function(cleaned_improved_solution):
            final_solution = improved_solution
        else:
            final_solution = proposed_solution
        
        # save and log the results
        completions.append( {'task_id': task_id, 'completion': final_solution} )
        write_jsonl(results_file, completions)
        logging.info('NEW REFLECTION PROMPT:\n' + new_full_prompt + '\n')
        logging.info('IMPROVED COMPLETION:\n' + final_solution + '\n')
        logging.info(f"Time elapsed including backoff: {(time.time() - start_time):.4f} seconds\n")

2025-06-16 04:35:53,458  INFO  BigCodeBench/0
2025-06-16 04:35:55,806  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:01,106  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:01,120  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
import itertools
from random import shuffle

def task_func(numbers=list(range(1, 3))):
    """
    Calculates the average of the sums of absolute differences between each pair of consecutive numbers 
    for all permutations of a given list. Each permutation is shuffled before calculating the differences.

    Args:
    - n

2025-06-16 04:36:06,011  INFO  Time elapsed including backoff: 4.8852 seconds

2025-06-16 04:36:06,012  INFO  BigCodeBench/4
2025-06-16 04:36:08,786  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:11,241  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:11,247  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
from collections import Counter
import itertools

def task_func(d):
    """
    Count the occurrence of each integer in the values of the input dictionary, where each value is a list of integers,
    and return a dictionary with these counts. The

2025-06-16 04:36:34,911  INFO  Time elapsed including backoff: 23.6610 seconds

2025-06-16 04:36:34,911  INFO  BigCodeBench/10
2025-06-16 04:36:37,765  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:40,324  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 04:36:40,338  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
import numpy as np
import itertools
import random
import statistics

def task_func(T1, RANGE=100):
    """
    Convert elements in 'T1' to integers and create a list of random integers.
    The size of the list is the sum of the integers in `T1

2025-06-16 04:36:56,619  INFO  IMPROVED COMPLETION:
The proposed solution is mostly correct, but there are a few minor issues and inefficiencies:

1. The function does not handle the case where the CSV file is empty. It should return an empty list in this case.
2. The function does not check if the output directory is writable. If it's not, the function should raise a permission error.
3. The function does not handle the case where the output file already exists. It should either overwrite the file or raise a file already exists error.
4. The function does not handle the case where the output file cannot be written. It should raise a permission error in this case.

Here's an optimized version of the proposed solution that addresses these issues:

```python
import subprocess
import csv
import os

def task_func(commands_file_path, output_dir_path):
    if not os.path.isfile(commands_file_path):
        raise FileNotFoundError(f"The file {commands_file_path} does not exist.")

    if not 

2025-06-16 04:37:12,530  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and does not contain any errors, inefficiencies, or inconsistencies. It correctly uses the `platform` and `psutil` modules to gather the required system information and calculates the memory usage percentage accurately. The function also returns the results in the expected dictionary format with the specified keys.

Here's the proposed solution again for reference:

```python
import psutil
import platform

def task_func():
    """
    Obtain system details, including operating system, architecture, and memory usage.

    This function gathers information about the system's operating system, architecture,
    and memory usage. It calculates the percentage of used memory by comparing the total
    and currently used memory. The gathered details are then returned in a dictionary
    format with specific keys for each piece of information.

    Returns:
    dict: A dictionary containing:
        - 

...

2025-06-16 08:28:48,247  INFO  Time elapsed including backoff: 10.2149 seconds

2025-06-16 08:28:48,248  INFO  BigCodeBench/1130
2025-06-16 08:28:50,666  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 08:28:55,387  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 08:28:55,412  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
import os
import hashlib
import json
from pathlib import Path

def task_func(directory: str) -> str:
    """
    Create SHA256 hashes for all files in the specified directory, including files in subdirectories, 
    and save these hashes in a

2025-06-16 08:29:02,991  INFO  IMPROVED COMPLETION:
The proposed solution is generally correct and efficient. However, there are a few potential issues and improvements that can be made:

1. **Error Handling**: The proposed solution does not handle potential errors that might occur during the database operations (e.g., if the user table does not exist, or if there is an issue with the database connection). It's a good practice to use try-except blocks to handle such errors gracefully.

2. **Efficiency**: The proposed solution retrieves and hashes each password one by one, which can be inefficient for large databases. A more efficient approach would be to use a single SQL command to update all passwords at once.

3. **Security**: The proposed solution retrieves the passwords one by one, which could potentially expose sensitive data if not handled properly. It's generally recommended to avoid retrieving sensitive data like passwords unless absolutely necessary.

Here's an optimized versi

2025-06-16 08:29:16,961  INFO  IMPROVED COMPLETION:
The proposed solution is already well-structured and efficient. It handles potential errors during API data fetching and file writing, and it correctly generates the filename based on the provided parameters. However, there's a minor improvement that can be made to enhance the error handling.

Currently, the function raises a `RuntimeError` with a generic error message when there's an issue fetching data from the API or writing to the file. While this is sufficient, it would be more helpful to include the specific error message from the exception that was raised. This would provide more context about what went wrong and could aid in debugging.

Here's the optimized version of the proposed solution with this improvement:

```python
import json
import requests

def task_func(API_URL, endpoint, PREFIX):
    """
    Retrieves JSON data from a specified API endpoint and writes it to a file. The filename is generated by concatenating
    a 

2025-06-16 08:29:22,916  INFO  Time elapsed including backoff: 5.9543 seconds

2025-06-16 08:29:22,917  INFO  BigCodeBench/1137
2025-06-16 08:29:25,484  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 08:29:30,205  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-16 08:29:30,231  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
import bs4
import requests
import re
import json

def task_func(url: str, output_path: str) -> list:
    """
    Extracts phone numbers from a given URL or local file and saves them to a specified file in JSON format.

    Parameters:
    - ur

2025-06-16 08:29:34,835  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and does not have any errors, inefficiencies, or inconsistencies. It correctly flattens the input matrix, sorts the elements in ascending order, and finds all unique combinations of two elements from the sorted array. The function also returns the results in the expected format as a tuple containing the sorted array and a list of tuples representing the combinations.

Here's the proposed solution again for reference:

```python
import numpy as np
import itertools

def task_func(matrix):
    """
    Sorts a numeric 2D numpy array in ascending order and finds all unique combinations of two elements from the sorted array.

    Parameters:
    - matrix (numpy.array): A 2D numpy array of any shape (m, n), where m and n are non-negative integers.

    Returns:
    - tuple: A tuple containing two elements:
        1. numpy.array: A 1D array with all elements of the input array sorted in ascending orde

In [10]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)